In [12]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import tkinter.filedialog as tkf
import hyperspy.api as hys

In [9]:
# refer to https://scipy-cookbook.readthedocs.io/items/FittingData.html

def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape) # row, col
    x = (X*data).sum()/total # row
    y = (Y*data).sum()/total # col
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum())
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum())
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

In [10]:
def gaussian_center(image, cbox_edge=0):
    y, x = np.indices(image.shape)
    if not cbox_edge:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
        
    else:
        cbox_outy = int(image.shape[0]/2 - cbox_edge/2)
        cbox_outx = int(image.shape[1]/2 - cbox_edge/2)
        center_box = image[cbox_outy:-cbox_outy, cbox_outx:-cbox_outx]
        fit_params = fitgaussian(center_box)
        (_, center_y, center_x, _, _) = fit_params
        center = [center_y+cbox_outy, center_x+cbox_outx]
        
    return center

In [2]:
def load_binary_4D_stack(img_adr, datatype, original_shape, final_shape, log_scale=True):
    raw_stack = np.fromfile(img_adr, dtype=datatype)
    stack = raw_stack.reshape(original_shape)
    print(stack.shape)
    if log_scale:
        stack = np.log(stack[:final_shape[0], :final_shape[1], :final_shape[2], :final_shape[3]])
    else:
        stack = stack[:final_shape[0], :final_shape[1], :final_shape[2], :final_shape[3]]
    
    print(stack.shape) 
    return stack

In [33]:
raw_adr = tkf.askopenfilename()
print(raw_adr)

E:/data_temporary/DGIST 20190925/G3_1_5/scan_x256_y256.raw


In [4]:
datatype = "float32"
o_shape = (256, 256, 130, 128)
f_shape = (256, 256, 128, 128)

In [34]:
stack_4d = load_binary_4D_stack(raw_adr, datatype, o_shape, f_shape, log_scale=False)
stack_4d = stack_4d / np.max(stack_4d) # normalize absolutely
stack_4d = stack_4d * 1E3
print(np.max(stack_4d))
print(np.min(stack_4d))
print(np.mean(stack_4d))
#print(np.median(stack_4d))

(256, 256, 130, 128)
(256, 256, 128, 128)
1000.0
1.1293406
5.227104


In [35]:
%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(np.sum(stack_4d, axis=(2, 3)), cmap="gray")

def onselect(eclick, erelease):
    print('startposition: (%f, %f)' % (eclick.xdata, eclick.ydata))
    print('endposition  : (%f, %f)' % (erelease.xdata, erelease.ydata))

box = RectangleSelector(ax, onselect)
plt.show()

startposition: (106.909111, 0.116053)
endposition  : (180.232766, 255.242195)


In [36]:
stack_4d_cropped = stack_4d[int(box.corners[1][0]):int(box.corners[1][2]), 
                                           int(box.corners[0][0]):int(box.corners[0][1])].copy()
print(stack_4d_cropped.shape)
print(np.max(stack_4d_cropped))
print(np.min(stack_4d_cropped))
print(np.mean(stack_4d_cropped))
#print(np.median(stack_4d_cropped))

(255, 74, 128, 128)
434.61038
1.1718267
4.387303


In [37]:
center_pos = []
for i in range(stack_4d_cropped.shape[0]):
    for j in range(stack_4d_cropped.shape[1]):
        center_pos.append(gaussian_center(stack_4d_cropped[i, j], cbox_edge=30))
        
center_pos = np.asarray(center_pos)
center_pos = np.reshape(center_pos, (stack_4d_cropped.shape[0], stack_4d_cropped.shape[1], -1))
center_mean = np.mean(center_pos, axis=(0, 1))
print(center_mean)

[64.45255178 64.95754504]


In [38]:
# save
stack_4d_cropped /= np.max(stack_4d_cropped)
stack_4d_cropped *= 100
save = hys.signals.Signal2D(stack_4d_cropped)
save.metadata.General.set_item("notes", center_mean.tolist())
save.save(tkf.asksaveasfilename())

In [ ]:
%matplotlib qt
interested.plot()

In [ ]:
print(stack_4d[0, 0])
print(np.max(stack_4d[0, 0]))
print(np.min(stack_4d[0, 0]))
print(np.mean(stack_4d[0, 0]))
print(np.median(stack_4d[0, 0]))
print(np.std(stack_4d[0, 0]))

In [ ]:
one_dif = stack_4d[128, 128]
one_dif = one_dif / np.max(one_dif)
one_dif[np.where(one_dif > 0.05)] = 0
plt.figure(figsize=(10,10))
plt.imshow(one_dif)
plt.show()

In [ ]:
print(one_dif)
print(np.max(one_dif))
print(np.min(one_dif))
print(np.mean(one_dif))
print(np.median(one_dif))
print(np.std(one_dif))